# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117920e+02     1.731046e+00
 * time: 0.08718419075012207
     1     1.094231e+01     1.008072e+00
 * time: 0.9430320262908936
     2    -1.176795e+01     1.167570e+00
 * time: 1.095870018005371
     3    -3.395608e+01     9.275373e-01
 * time: 1.2343530654907227
     4    -4.750550e+01     6.511170e-01
 * time: 1.3861801624298096
     5    -5.671711e+01     2.701177e-01
 * time: 1.5302481651306152
     6    -5.945605e+01     2.799139e-01
 * time: 1.6349921226501465
     7    -6.061725e+01     1.520144e-01
 * time: 1.7454791069030762
     8    -6.117942e+01     7.012199e-02
 * time: 1.8517100811004639
     9    -6.146611e+01     5.182817e-02
 * time: 1.9628381729125977
    10    -6.172059e+01     3.933205e-02
 * time: 2.068446159362793
    11    -6.188813e+01     2.605729e-02
 * time: 2.17799711227417
    12    -6.201710e+01     2.094150e-02
 * time: 2.2829701900482178
    13    -6.207906e+01     1.645206e-02
 * time: 2.3928990364074

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557610
    AtomicNonlocal      14.8522591
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485356 
    Xc                  -19.3336815

    total               -62.261666460550
